<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/tam1444AH/UH-Insure-NSA.git"
REPO="UH-Insure-NSA"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

# save test.

Cloning repo 'UH-Insure-NSA'...
Cloning into 'UH-Insure-NSA'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 271 (delta 112), reused 11 (delta 11), pack-reused 89 (from 1)
Receiving objects: 100% (271/271), 487.01 KiB | 9.19 MiB/s, done.
Resolving deltas: 100% (138/138), done.
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 110.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 40.1 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [2]:
from model.test import test
test()

Hello World!


In [3]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

Logged in as: tam2003


In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
os.chdir("/content")

In [6]:
import json, hashlib

src = "/content/verified_GPThybrid.jsonl"
out = "/content/hybrid_tagged_unsupervised.jsonl"

def sha1_of(text):
    return hashlib.sha1(text.encode("utf-8")).hexdigest()[:10]

with open(out, "w") as f_out, open(src, "r") as f_in:
  for line in f_in:
    try:
      data = json.loads(line)
      filetype = data.get("filetype", "").lower()
      filename = data.get("filename", "unknown")
      content = data.get("content", "").strip()
      variant = data.get("variant", "hybrid")

      if filetype == "cry" or filename.endswith(".cry"):
          tag = "cryptol"
      elif filetype == "saw" or filename.endswith(".saw"):
          tag = "saw"
      else:
          tag = "unknown"

      sha = sha1_of(content)
      wrapped = (
          f"<FILE:{tag}> <PATH:{filename}> <VARIANT:{variant}> <SHA:{sha}>\n"
          f"<CODE>\n{content}\n</CODE>\n</FILE>\n"
      )
      f_out.write(json.dumps({"content": wrapped}) + "\n")
    except Exception as e:
      print("skip:", e)

In [7]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen2.5-Coder-32B"  # Model checkpoint on the Hugging Face Hub
DATASET = "/content/hybrid_tagged_unsupervised.jsonl" # Josh's preproccesed dataset.
DATA_COLUMN = "content"  # Column name containing the code content

SEQ_LENGTH = 2048  # Sequence length

# Training arguments
# MAX_STEPS = 1#2000  # max_steps
MAX_STEPS = 200#2000  # max_steps
BATCH_SIZE = 1  # batch_size
# GR_ACC_STEPS = 16  # gradient_accumulation_steps
GR_ACC_STEPS = 8  # gradient_accumulation_steps
# LR = 5e-4  # learning_rate
LR = 2e-4                     # a bit lower for stability
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.05  # weight_decay
# WEIGHT_DECAY = 0.01  # weight_decay
NUM_WARMUP_STEPS = 100  # num_warmup_steps
# NUM_WARMUP_STEPS = 30  # num_warmup_steps
# EVAL_FREQ = 200
EVAL_FREQ = 50
SAVE_FREQ = 200
LOG_FREQ = 10
LOG_FREQ = 50
# EVAL_FREQ = 100  # eval_freq
# SAVE_FREQ = 100  # save_freq
# LOG_FREQ = 25  # log_freq
OUTPUT_DIR = "peft-FT-2.5-Coder-32b"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.5  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate

# LORA
LORA_R = 8  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = "c_proj,c_attn,q_attn,c_fc,c_proj"  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 0

In [8]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)

set_seed(SEED)

In [11]:
from datasets import load_dataset

DATASET = "/content/hybrid_tagged_unsupervised.jsonl"
DATA_COLUMN = "content"

dataset = load_dataset("json", data_files={"train": DATASET})
train_data = dataset["train"]

print(train_data[0].keys())
assert DATA_COLUMN in train_data.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"


dict_keys(['content'])


In [12]:
import torch
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

def chars_token_ratio(dataset, tokenizer, data_column, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """

    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        total_characters += len(example[data_column])
        total_tokens += len(tokenizer(example[data_column]).tokens())

    return total_characters / total_tokens


chars_per_token = chars_token_ratio(train_data, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

100%|██████████| 400/400 [00:01<00:00, 246.30it/s]

The character to token ratio of the dataset is: 2.20


In [13]:
from model.dataset.util import *

In [14]:
from model.dataset.constantLengthDataset import ConstantLengthDataset


train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        infinite=True,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)
eval_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        #valid_data,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)

In [15]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

load_in_8bit = False

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

device_map = "auto"#{"": 0}

base = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
        use_cache=False,  # We will be using gradient checkpointing
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
        offload_folder="/content/offload"
)

model = prepare_model_for_kbit_training(base)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00005-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/139 [00:00<?, ?B/s]

In [16]:
blk = model.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

ATTN: Qwen2Attention(
  (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
  (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
  (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
  (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
)
MLP: Qwen2MLP(
  (gate_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
  (up_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
  (down_proj): Linear4bit(in_features=27648, out_features=5120, bias=False)
  (act_fn): SiLU()
)


In [18]:
# Set up lora
from peft import PeftModel

peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = PeftModel.from_pretrained(base, "tam2003/peft-FT-2.5-Coder-32b")
model = get_peft_model(base, peft_config)

model.print_trainable_parameters()

adapter_config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

trainable params: 0 || all params: 32,830,985,216 || trainable%: 0.0000


In [ ]:
train_data.start_iteration = 0

training_args = TrainingArguments(
    output_dir=f"tam1444AH/{OUTPUT_DIR}",
    dataloader_drop_last=True,
    eval_strategy="steps",
    save_strategy="steps",
    max_steps=MAX_STEPS,
    eval_steps=EVAL_FREQ,
    save_steps=SAVE_FREQ,
    logging_steps=LOG_FREQ,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=NUM_WARMUP_STEPS,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=True,
    fp16=FP16,
    bf16=BF16,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=True,
    include_tokens_per_second=True,
    report_to=[],
    resume_from_checkpoint=True,
)

print(f"Training samples: {len(train_data)}")
print(f"Sample text:\n\n{train_data[0][DATA_COLUMN][:400]}")

In [ ]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

print("Training...")
trainer.train(resume_from_checkpoint=True)

In [ ]:
trainer.push_to_hub()

In [ ]:
!pip install -q transformers peft
!pip install -q transformers peft bitsandbytes accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Base model load
base = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Coder-32B",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA fine-tuned adapter on top of it
model = PeftModel.from_pretrained(base, "tam2003/peft-FT-2.5-Coder-32b")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-32B", trust_remote_code=True)
model.eval()


In [ ]:
from transformers import pipeline
import gc, torch
gc.collect()
torch.cuda.empty_cache()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="balanced_low_0",
    dtype="bfloat16",
)

prompt = """
Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input block and returns a 256-bit digest."
"""
result = pipe(prompt, max_new_tokens=300)
print(result[0]["generated_text"])

prompts = [
    'Translate this English spec into Cryptol:\n\n"A function xor_inverse that proves (a XOR b) XOR b == a"',
    'Translate this English spec into Cryptol:\n\n"A function sha256_hash that takes a 512-bit input and returns a 256-bit digest"',
    'Translate this English spec into Cryptol:\n\n"Create a property proving that addition is commutative for 8-bit words."'
]

for p in prompts:
    print("\n=== Prompt ===")
    print(p)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    print("\n--- Generated Cryptol ---")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
prompt = """
Translate this English spec into SAW script:

"Verify that the function xor_inverse in xor_inverse.c returns (a XOR b) XOR b == a
for all 8-bit values of a and b."
"""
result = pipe(prompt, max_new_tokens=400)
print(result[0]["generated_text"])

prompt = """
Generate both Cryptol and SAW verification code for this spec:
"A function add8 that proves addition is commutative for 8-bit words."
"""
result = pipe(prompt, max_new_tokens=400)
print(result[0]["generated_text"])

prompt = """
Write a SAW script verifying that the Cryptol function sha256_hash implements
the same behavior as the C function sha256_hash_ref defined in sha256.c
using crucible_llvm_verify.
"""

result = pipe(prompt, max_new_tokens=400)
print(result[0]["generated_text"])



In [ ]:
!apt-get update -q
!apt-get install -y wget tar

!wget -q https://github.com/GaloisInc/saw-script/releases/download/v1.3/saw-1.3-ubuntu-22.04-X64-with-solvers.tar.gz
!tar -xzf saw-1.3-ubuntu-22.04-X64-with-solvers.tar.gz
!mv saw-1.3-ubuntu-22.04-X64-with-solvers saw

!wget -q https://github.com/GaloisInc/cryptol/releases/download/3.3.0/cryptol-3.3.0-ubuntu-20.04-X64-with-solvers.tar.gz
!tar -xzf cryptol-3.3.0-ubuntu-22.04-X64-with-solvers.tar.gz
!mv cryptol-3.3.0-ubuntu-22.04-X64-with-solvers cryptol

In [ ]:
import os
os.environ["PATH"] = "/content/saw/bin:/content/cryptol/bin:" + os.environ["PATH"]

In [ ]:
!saw --version
!cryptol --version

In [ ]:
!git clone https://github.com/GaloisInc/cryptol-specs.git

In [ ]:
%cd /content/cryptol-specs/Primitive/Symmetric/Cipher/Block/AES/Verifications
os.environ["CRYPTOLPATH"] = "/content/cryptol-specs"

In [ ]:
!saw AES128.saw

In [ ]:
%cd /content
!git clone https://github.com/plobethus/crypto-c.git

In [ ]:
%cd crypto-c

In [ ]:
!chmod +x scripts/colab_setup.sh
!bash scripts/colab_setup.sh
%cd /content